In [2]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [3]:
from pdf2image import convert_from_path
import pytesseract
import cv2
from pytesseract import Output
import pandas as pd
import numpy as np

In [4]:
def confidenceLevelSegregation(df,confLevel):
    #Lower confidence boxes shown in red
    if int(df['conf']) < confLevel:
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'Low'
    else: # Rest of the boxes are in green
        (x, y, w, h) = (df['left'], df['top'], df['width'], df['height'])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0,0, 255), 2) #(BGR),thickness
        df['confidenceLevel'] = 'High'
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    # df['confidenceLevel'] = np.where(
    #     df['conf'] < confLevel, # condition
    #     "Low",    # result if true
    #     "High",   # result if false
    # )
    # df['confidenceLevel'] = df['conf'].map(lambda x : "Low" if(x < confLevel) else "High")
    return df

In [15]:
def paraBreakdownConfidenceLevel(folderName,pageNum,confLevel):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum

    for i in range(len(grouped_df)):
        if int(grouped_df.loc[i,'left']) < confLevel:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            grouped_df['confidenceLevel'] = 'Low'
        else:
            (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
            img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, ), 2)
            grouped_df['confidenceLevel'] = 'High'
            
    # # Creating a visualization that showcases the boxes 
    # # cv2.imshow('img', img)
    # # cv2.waitKey(5000)
    return grouped_df


    # segregatedDF = confidenceLevelSegregation(grouped_df,confLevel)
    # return segregatedDF


In [16]:
def paraBreakdown(folderName,pageNum):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum
    # # Creating a visualization that showcases the boxes 
    # for i in range(len(grouped_df)):
    # # print(grouped_df.loc[i,'left'])
    #     if int(grouped_df.loc[i,'left']) > 60:
    #         (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    #         img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # cv2.imshow('img', img)
    # cv2.waitKey(5000)
    # grouped_df.to_csv(f'./Page_{pageNum}paraBlocksTrial.csv')

    # display(grouped_df)
    return grouped_df

In [19]:
#Converting pdf pages into individual images
def convertPDFtoImages(pdfPath,pdfFolderToStoreImages,outputFilePath,confLevel):
    pdfs = pdfPath #"./Articles/amz_letters.pdf"
    pages = convert_from_path(pdfs, 350)
    print('Ingested all the pages from the pdf. Moving on to process them next ==>')
    df = pd.DataFrame()
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(f'{pdfFolderToStoreImages}/{image_name}', "JPEG")
        print(f'Doing iteration number {i}')
        res = paraBreakdownConfidenceLevel(pdfFolderToStoreImages,i,confLevel)
        # display(res)
        df=pd.concat([df,res], ignore_index=True)
        i = i+1
    display(df)
    df.to_csv(f'{outputFilePath}')

In [20]:
convertPDFtoImages('./Articles/Carbon-Credit-Carbon-Offset-Fundamentals.pdf','carbonCredit','carbonCredit/fullCarbonCreditParaBlocks.csv',30)

Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27


,block_num,left,top,width,height,conf,text,Page,confidenceLevel
0,0,0,0,2975,3850,-1.000000,,1,Low
1,1,265,228,2448,481,52.142857,CARBON CREDIT l | AND CARBON OFFSET MINTZ...,1,Low
2,2,260,216,394,328,23.000000,,1,Low
3,3,860,792,434,20,23.000000,,1,Low
4,4,862,895,1297,180,66.357143,"Areta A. Jez, Brad D. Alexander, and Ayaz ...",1,Low
...,...,...,...,...,...,...,...,...,...
413,3,287,977,544,117,23.000000,MINTZ,27,High
414,4,0,0,832,1098,23.000000,,27,High
415,5,2171,3393,534,76,20.250000,mintz.com,27,High
416,6,188,3602,2519,54,78.150000,Boston | London | Los Angeles | New York | ...,27,High


In [21]:
convertPDFtoImages('./Articles/putting-carbon-markets-to-work-on-the-path-to-net-zero.pdf','carbonNetZero','carbonNetZero/fullCarbonNetZeroParaBlocks.csv',30)

Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27
Doing iteration number 28
Doing iteration number 29
Doing iteration number 30
Doing iteration number 31
Doing iteration number 32
Doing iteration number 33
Doing iteration number 34
Doing iteration number 35
Doing iteration number 36
D

,block_num,left,top,width,height,conf,text,Page,confidenceLevel
0,0,0,0,2894,4093,-1.000000,,1,High
1,1,197,1116,487,65,37.800000,October 2021,1,High
2,2,600,3869,255,31,23.250000,SINGAPORE,1,High
3,3,194,3758,1055,159,44.571429,ii GIC EDB: McKinsey,1,High
4,4,1039,3845,369,72,26.800000,& Company,1,High
...,...,...,...,...,...,...,...,...,...
1308,2,193,3537,46,151,22.666667,vy in,84,High
1309,3,193,3785,42,29,22.250000,in,84,High
1310,4,370,3534,974,163,53.000000,@MckKinsey / @EDBsg / @gicsingapore @MckKi...,84,High
1311,5,371,3714,1553,109,19.500000,https://sg.linkedin.com/company/singapore-e...,84,High


In [22]:
convertPDFtoImages('./Articles/MethodologyForAssessingTheQualityOfCarbonCredits.pdf','carbonMethodology','carbonMethodology/fullcarbonMethodologyParaBlocks.csv',30)


Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27
Doing iteration number 28
Doing iteration number 29
Doing iteration number 30
Doing iteration number 31
Doing iteration number 32
Doing iteration number 33
Doing iteration number 34
Doing iteration number 35
Doing iteration number 36
D